<a href="https://colab.research.google.com/github/estesoymio/DSPortfolio/blob/main/StreamlitApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploy on Streamlit

In [9]:
import streamlit as st
import boto3
import tensorflow as tf
from PIL import Image
import numpy as np
import os



In [13]:
# AWS S3 credentials
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
BUCKET_NAME = os.getenv('BUCKET_NAME')
MODEL_PATH = 'Cat_dog_model.h5'


In [14]:
# Download model from S3
def download_model():
    try:
        s3 = boto3.client(
            's3',
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY
        )
        if not os.path.exists(MODEL_PATH):
            st.write("Downloading model from S3...")
            with open(MODEL_PATH, 'wb') as f:
                s3.download_fileobj(BUCKET_NAME, MODEL_PATH, f)
        else:
            st.write("Model already exists locally.")
        return MODEL_PATH
    except Exception as e:
        st.error(f"Error downloading model: {e}")
        raise e

# Load the model
try:
    model_path = download_model()
    if os.path.exists(model_path):
        st.write(f"Loading model from {model_path}")
        model = tf.keras.models.load_model(model_path)
        st.write("Model loaded successfully.")
    else:
        st.error(f"Model file does not exist at {model_path}")
except Exception as e:
    st.error(f"Error loading model: {e}")

OSError: Unable to open file (file signature not found)

In [ ]:
# Define the prediction function
def predict_image(image):
    try:
        image = image.resize((224, 224))
        image_array = np.array(image) / 255.0
        image_array = np.expand_dims(image_array, axis=0)
        prediction = model.predict(image_array)
        return prediction[0]
    except Exception as e:
        st.error(f"Error predicting image: {e}")


In [ ]:
# Define the Streamlit app
st.title('Image Prediction Model Deployment with Streamlit')

# Collect user input: Image upload
st.header('Upload an Image')
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)

    if st.button('Predict'):
        result = predict_image(image)
        st.write(f'Prediction: {result}')